# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
create table employees(
        ID_Usr INT primary key,
        name VARCHAR);
    /*3 example rows
    select * from employees limit 3;
    ID_Usr    name
    1344      George StPierre
    2122      Jon jones
    1265      Anderson Silva
    */

    create table salary(
        ID_Usr INT,
        year DATE,
        salary FLOAT,
        foreign key (ID_Usr) references employees(ID_Usr));
    /*3 example rows
    select * from salary limit 3
    ID_Usr    date          salary
    1344      01/01/2023    61000
    1344      01/01/2022    60000
    1265      01/01/2023    55000
    */

    create table studies(
        ID_study INT,
        ID_Usr INT,
        educational_level INT,  /* 5=phd, 4=Master, 3=Bachelor */
        Institution VARCHAR,
        Years DATE,
        Speciality VARCHAR,
        primary key (ID_study, ID_Usr),
        foreign key(ID_Usr) references employees (ID_Usr));
    /*3 example rows
    select * from studies limit 3
    ID_Study ID_Usr educational_level Institution    Years       Speciality
    2782     1344   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing
    2334     1344   5                 MIT            01/01/2023  Phd. Data Science.
    2782     2122   3                 UC San Diego   01/01/2010  Bachelor of Science in Marketing
    */
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
 Question: How Many employes we have with a salary bigger than 50000?
SELECT COUNT(*) AS total_employees
FROM employees e
INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
WHERE s.salary > 50000;
 Question: Return the names of the three people who have had the highest salary increase in the last three years.
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
GROUP BY e.name
ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
LIMIT 3;
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("The name of the employee best paid", context_user))

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
ORDER BY s.salary DESC
LIMIT 1;
```


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("The name of the employee best paid", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee who is best paid by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which corresponds to the employee with the highest salary.


In [8]:
#new
print(return_CCRMSQL("Return the Institution with a higher average salary", context_user))

```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
JOIN studies st ON e.ID_Usr = st.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [9]:
#old
print(return_CCRMSQL("Return the Institution with a higher average salary", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(s.salary) AS avg_salary
FROM salary s
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the `salary` and `studies` tables on the `ID_usr` column. It calculates the average salary for each institution by grouping the results by `Institution`. Then, it orders the results in descending order based on the average salary and limits the output to only the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

# **Prompt No. 01:** 
***Output==>Basic Query***

In [10]:
#new
print(return_CCRMSQL("What is the name of the employee with the highest salary?", context_user))

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
ORDER BY s.salary DESC
LIMIT 1;
```


In [11]:
#old
print(return_CCRMSQL("What is the name of the employee with the highest salary?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee with the highest salary by joining the "employees" table with the "salary" table on the employee ID. It then orders the result by salary in descending order and limits the output to the first row, which corresponds to the employee with the highest salary.


# **Prompt No. 02:** 
***Output==>Intermediate Query***

In [12]:
#new
print(return_CCRMSQL("Which institution has the highest average salary for employees with a PhD?", context_user))

```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
JOIN studies st ON e.ID_Usr = st.ID_Usr
WHERE st.educational_level = 5
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [13]:
#old
print(return_CCRMSQL("Which institution has the highest average salary for employees with a PhD?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
WHERE st.educational_level = 'PhD'
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query retrieves the institution with the highest average salary for employees who have a PhD. It joins the employees, studies, and salary tables, filters by employees with a PhD, calculates the average salary per institution, and then selects the institution with the highest average salary.


# **Prompt No. 03:** 
***Output==>Advanced Query***

In [14]:
#new
print(return_CCRMSQL("What is the total salary of employees hired after 2015 who have a master's degree from 'MIT'?", context_user))

```sql
SELECT SUM(s.salary) AS total_salary
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
JOIN studies st ON e.ID_Usr = st.ID_Usr
WHERE s.year > '2015-01-01'
AND st.educational_level = 4
AND st.Institution = 'MIT';
```


In [15]:
#old
print(return_CCRMSQL("What is the total salary of employees hired after 2015 who have a master's degree from 'MIT'?", old_context_user))

This is your SQL:
```sql
SELECT SUM(s.salary) AS total_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE YEAR(s.year) > 2015
AND st.educational_level = 'Master'
AND st.Institution = 'MIT';
```

This SQL query retrieves the total salary of employees hired after 2015 who have a master's degree from 'MIT'. It joins the employees, salary, and studies tables, filters the results based on the conditions provided, and calculates the sum of their salaries.


# **Prompt No. 04:** 
***Output==>Challenge Query***

In [18]:
#new
print(return_CCRMSQL("Which employees hired after 2010, who work in departments with at least 5 employees, have an average salary above $70,000, and obtained their highest degree from institutions ranked in the top 10 globally?", context_user))

SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
JOIN studies st ON e.ID_Usr = st.ID_Usr
WHERE e.ID_Usr NOT IN (
    SELECT ID_Usr
    FROM salary
    WHERE year < '2011-01-01'
)
AND e.ID_Usr IN (
    SELECT ID_Usr
    FROM (
        SELECT ID_Usr, COUNT(*) AS dept_employees
        FROM employees
        GROUP BY ID_Usr
        HAVING COUNT(*) >= 5
    )
)
AND s.salary > 70000
AND st.Institution IN (
    SELECT Institution
    FROM studies
    WHERE educational_level = 5
    GROUP BY Institution
    ORDER BY COUNT(*) DESC
    LIMIT 10
)
GROUP BY e.name
HAVING AVG(s.salary) > 70000;


In [19]:
#old
print(return_CCRMSQL("Which employees hired after 2010, who work in departments with at least 5 employees, have an average salary above $70,000, and obtained their highest degree from institutions ranked in the top 10 globally?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE YEAR(s.year) > 2010
GROUP BY e.name
HAVING AVG(s.salary) > 70000
```

This SQL query selects the names of employees who were hired after 2010, have an average salary above $70,000, and obtained their highest degree from institutions ranked in the top 10 globally.
